In [1]:
from dolfinx import io, fem, mesh as msh
from Training_utils import test_set
from SPDE_problems import *
from FEniCSx_solver import plot_fn
import torch
import torch_geometric as tg
from SPDE_problems import Data_to_solver
from SUPG_prediction_models import *
from FEniCSx_solver import interpolate_expr


mlp, gcn, sage, gat, gatv2 = MLP(), GCN(), SAGE(), GAT(), GATv2()

mlp_state = torch.load("data/models/MLP.pth")
mlp.load_state_dict(mlp_state['model_state'])
gcn_state = torch.load("data/models/GCN.pth")
gcn.load_state_dict(gcn_state['model_state'])
sage_state = torch.load("data/models/SAGE.pth")
sage.load_state_dict(sage_state['model_state'])
gat_state = torch.load("data/models/GAT.pth")
gat.load_state_dict(gat_state['model_state'])
gatv2_state = torch.load("data/models/GATv2.pth")
gatv2.load_state_dict(gatv2_state['model_state'])


fs , G = Data_to_solver(0, train = False)

/Users/vincentdallmer/dev/MA_ML_assisted_SUPG/FEniCSx_solver.py:86: SparseEfficiencyWarning: splu converted its input to CSC format
  A_inv = scipy.sparse.linalg.splu(self._A_scipy)


array([ 0.5      , -0.8660254,  0.5      , ..., -0.8660254,  0.5      ,
       -0.8660254], shape=(4096,))

In [20]:
b2.x.array

array([-0.8660254, -0.8660254, -0.8660254, ..., -0.8660254, -0.8660254,
       -0.8660254], shape=(2048,))

In [ ]:

from mpi4py import MPI
for num in range(14):
    G = test_set[num]
    fs, G = Data_to_solver(num, train=False)
    mesh = fs.domain
    mesh.anem = 'mesh'
    V = fs.uh.function_space
    Y = fs.Yh
    B = fem.functionspace(mesh=mesh, element=("P", 1, (2,)))
    b = fem.Function(B)

    b1=interpolate_expr(fs.b[0], V)
    b2=interpolate_expr(fs.b[1], V)
    b.sub(0).interpolate(b1)
    b.sub(1).interpolate(b2)
    b.name = 'b'

    eps, c, f, g = interpolate_expr(fs.eps,V), interpolate_expr(fs.c,V), interpolate_expr(fs.f,V), interpolate_expr(fs.g,V)
    try:
        uD = fs.uD
        uD.name = 'uD'
    except:
        UD = None
    eps.name = 'eps'
    c.name = 'c'
    f.name = 'f'
    g.name = 'g'

    u_opt, u_std, u_mlp, u_gcn, u_sage, u_gat, u_gatv2 = fem.Function(V), fem.Function(V), fem.Function(V), fem.Function(V), fem.Function(V), fem.Function(V), fem.Function(V)
    y_opt, y_std, y_mlp, y_gcn, y_sage, y_gat, y_gatv2 = fem.Function(Y), fem.Function(Y), fem.Function(Y), fem.Function(Y), fem.Function(Y), fem.Function(Y), fem.Function(Y)

    u_std.x.array[:] = fs.uh.x.array.copy()
    u_std.name = 'u_std'
    y_std.x.array[:] = fs.yh.x.array.copy()
    y_std.name = 'y_std'

    fs.set_weights(G.y.view(-1).detach().numpy())
    u_opt.x.array[:] = fs.uh.x.array.copy()
    u_opt.name = 'u_opt'
    y_opt.x.array[:] = fs.yh.x.array.copy()
    y_opt.name = 'y_opt'

    w = mlp(G)
    fs.set_weights(w.view(-1).detach().numpy())
    u_mlp.x.array[:] = fs.uh.x.array.copy()
    u_mlp.name = 'u_mlp'
    y_mlp.x.array[:] = fs.yh.x.array.copy()
    y_mlp.name = 'y_mlp'

    w = gcn(G)
    fs.set_weights(w.view(-1).detach().numpy())
    u_gcn.x.array[:] = fs.uh.x.array.copy()
    u_gcn.name = 'u_gcn'
    y_gcn.x.array[:] = fs.yh.x.array.copy()
    y_gcn.name = 'y_gcn'

    w = sage(G)
    fs.set_weights(w.view(-1).detach().numpy())
    u_sage.x.array[:] = fs.uh.x.array.copy()
    u_sage.name = 'u_sage'
    y_sage.x.array[:] = fs.yh.x.array.copy()
    y_sage.name = 'y_sage'
    
    w = gat(G)
    fs.set_weights(w.view(-1).detach().numpy())
    u_gat.x.array[:] = fs.uh.x.array.copy()
    u_gat.name = 'u_gat'
    y_gat.x.array[:] = fs.yh.x.array.copy()
    y_gat.name = 'y_gat'

    w = gatv2(G)
    fs.set_weights(w.view(-1).detach().numpy())
    u_gatv2.x.array[:] = fs.uh.x.array.copy()
    u_gatv2.name = 'u_gatv2'
    y_gatv2.x.array[:] = fs.yh.x.array.copy()
    y_gatv2.name = 'y_gatv2'

    with io.XDMFFile(MPI.COMM_WORLD, f"data/models//results/results_{num:03d}.xdmf", "w") as xdmf:
        xdmf.write_mesh(mesh)
        xdmf.write_function(u=eps)
        xdmf.write_function(u=b)
        xdmf.write_function(u=c)
        xdmf.write_function(u=g)
        xdmf.write_function(u=f)
        if uD:
            xdmf.write_function(u=uD)
        xdmf.write_function(u=u_std)
        xdmf.write_function(u=u_opt)
        xdmf.write_function(u=u_mlp)
        xdmf.write_function(u=u_gcn)
        xdmf.write_function(u=u_sage)
        xdmf.write_function(u=u_gat)
        xdmf.write_function(u=u_gatv2)
        xdmf.write_function(u=y_std)
        xdmf.write_function(u=y_opt)
        xdmf.write_function(u=y_mlp)
        xdmf.write_function(u=y_gcn)
        xdmf.write_function(u=y_sage)
        xdmf.write_function(u=y_gat)
        xdmf.write_function(u=y_gatv2)

/Users/vincentdallmer/dev/MA_ML_assisted_SUPG/FEniCSx_solver.py:86: SparseEfficiencyWarning: splu converted its input to CSC format
  A_inv = scipy.sparse.linalg.splu(self._A_scipy)
/Users/vincentdallmer/dev/MA_ML_assisted_SUPG/FEniCSx_solver.py:119: SparseEfficiencyWarning: splu converted its input to CSC format
  Adj_inv = scipy.sparse.linalg.splu(self._Adj_scipy)
